<h2> Loading the Data sets </h2>

In [47]:
import pandas as pd

mergedData = pd.read_csv("data/merged.csv")

mergedData['id'] = mergedData['id'].apply(str)

mergedData.head(10)

,id,content,date,metions,hashtags,in_reply_to_user_id_str,is_retweet,retweets,favorites
0,1698308935,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,NaN,NaN,NaN,False,501.0,879.0
1,1701461182,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,NaN,NaN,NaN,False,33.0,271.0
2,1737479987,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,NaN,NaN,NaN,False,12.0,20.0
3,1741160716,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,NaN,NaN,NaN,False,10.0,27.0
4,1773561338,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,NaN,NaN,NaN,False,1421.0,1954.0
5,1776419923,"Miss USA Tara Conner will not be fired - ""I've...",2009-05-12 14:21:55,NaN,NaN,NaN,False,28.0,28.0
6,1786560616,Listen to an interview with Donald Trump discu...,2009-05-13 12:38:28,NaN,NaN,NaN,False,14.0,17.0
7,1796477499,"""Strive for wholeness and keep your sense of w...",2009-05-14 11:30:40,NaN,NaN,NaN,False,17.0,27.0
8,1806258917,"Enter the ""Think Like A Champion"" signed book ...",2009-05-15 09:13:13,NaN,NaN,NaN,False,14.0,9.0
9,1820624395,"""When the achiever achieves, it's not a platea...",2009-05-16 17:22:45,NaN,NaN,NaN,False,19.0,49.0


In [48]:
mergedData.shape

(42295, 9)

In [4]:
qualityCheckData = pd.read_csv("data/qualityCheckData_2020-06-13.csv")

# change id column to string and make it the index
qualityCheckData['id'] = qualityCheckData['id'].apply(str)
qualityCheckData = qualityCheckData.set_index('id')

qualityCheckData.head(10)

,id_str,text,created_at,retweet_count,favorite_count
id,,,,,
1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 18:54:25+00:00,509,917
1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 01:00:10+00:00,34,269
1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 13:38:08+00:00,13,20
1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 20:40:15+00:00,11,27
1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 14:07:28+00:00,1376,1951
1776419923,https://twitter.com/realDonaldTrump/status/177...,"Miss USA Tara Conner will not be fired - ""I've...",2009-05-12 19:21:55+00:00,29,29
1786560616,https://twitter.com/realDonaldTrump/status/178...,Listen to an interview with Donald Trump discu...,2009-05-13 17:38:28+00:00,15,17
1796477499,https://twitter.com/realDonaldTrump/status/179...,"""Strive for wholeness and keep your sense of w...",2009-05-14 16:30:40+00:00,18,28
1806258917,https://twitter.com/realDonaldTrump/status/180...,"Enter the ""Think Like A Champion"" signed book ...",2009-05-15 14:13:13+00:00,15,10


In [49]:
qualityCheckData.shape

(42150, 5)

<h2> Comparing the content of the merged Data with the content of the data scraped at 2020-06-13. </h2>

In [41]:
from difflib import SequenceMatcher
import re

def mystrip(s):
    # get rid of any whitespaces, tabs, newlines and carriage returns
    # in front of and behind the string, just in case
    s = s.strip(' \t\n\r')
    s = re.sub(r"https?:\/\/\S+", "", s) #remove urls from a
    s = re.sub(r"pic\.twitter\.com\/\S+", "", s) #remove picute urls from b
    s = re.sub('[\s+]', '', s.lower() ) #strip whitespace inbetween words and characters
    return s

def similarity(a, b):
    return SequenceMatcher(None, mystrip(a), mystrip(b)).ratio()

ids_with_no_text = []
ids_with_no_match = []
ids_with_low_similarity = []
sum_similarities = 0.0
sum_retweetsDistance = 0
sum_favoritesDistance = 0
num_matches = 0

for index, row in mergedData.iterrows():
    if row['id'] in qualityCheckData.index:
        
        areStrings = isinstance(row['content'], str) and isinstance(qualityCheckData.at[row['id'], 'text' ], str)
        
        if not areStrings:
            
            ids_with_no_text.append(row['id'])
            continue
            
        tmpSimilarity = similarity( row['content'], qualityCheckData.at[row['id'], 'text' ] )
        
        if tmpSimilarity < 0.9:
            
            ids_with_low_similarity.append([row['id'],
                                            row['content'],
                                            qualityCheckData.at[row['id'], 'text'],
                                            tmpSimilarity])
        sum_similarities += tmpSimilarity
        sum_retweetsDistance += abs(row['retweets'] - qualityCheckData.at[row['id'], 'retweet_count' ])
        sum_favoritesDistance += abs(row['favorites'] - qualityCheckData.at[row['id'], 'favorite_count' ])
        num_matches += 1
        
    else:
        
        ids_with_no_match.append(row['id'])

average_similarity = sum_similarities / num_matches
average_retweetDistance = sum_retweetsDistance / num_matches
average_favoriteDistance = sum_favoritesDistance / num_matches


In [42]:
print("average similarity: ", average_similarity)
print("average retweet distance: ", average_retweetDistance)
print("average favorite distance: ", average_favoriteDistance)
print("Number of ids with similarity below 0.9: ", len(ids_with_low_similarity))
print("Number of matches: ", num_matches)
print("Number of tweets without text in the check Data: ", len(ids_with_no_text))
print("Number tweets without a match: ", len(ids_with_no_match))
print("Number of matches + number of tweets without text + number of tweets with no match: ",
      num_matches + len(ids_with_no_text) + len(ids_with_no_match))

average similarity:  0.997476479861023
average retweet distance:  614.3543933656105
average favorite distance:  1867.4073630219068
Number of ids with similarity below 0.9:  30
Number of matches:  41722
Number of tweets without text in the check Data:  426
Number tweets without a match:  147
Number of matches + number of tweets without text + number of tweets with no match:  42295


As we can see the two datasets are pretty similar. In the list of ids with similarity below 0.9 we can spot why we have a value of 0.95 and not 1. The Library I used to scrape the data for the quality test does not include URL's into the text body of the tweets. Another difference is that mentions have a white space character inbetween the name and the @-sign. To get around those problems I striped the text of any whitespace, urls and twitter picuter urls to compare them. As a result we get an average similarity of 0.99 which is pretty close. The last problem with comparing our scraped data with the data set from kaggle is the presentation of emojis.

In [30]:
print(ids_with_no_match)

['11366768299', '182888648600068096', '182889806953590784', '182891369541545984', '182894962738872322', '182904845047578624', '182908230412939264', '182920478413946880', '182921423575187457', '182922651944230912', '182924179660738560', '313449312367570944', '313455112855121920', '313455619287965697', '313456341412880384', '313456449181319169', '313456639581777921', '313457042935398400', '313457914570489856', '313459226053861377', '313459978348404738', '313462166999146496', '313463029251571712', '313463764966064128', '313464589608177664', '313466007320662016', '313467091120422912', '313469847898050560', '313472689820016640', '313667016554840065', '313669482012876800', '313676691815927808', '313711625595146241', '313718098064711680', '313718385148043264', '313725577783824385', '313729764714229760', '313730158869741568', '313730463153930240', '313734916737204224', '313735229430964227', '313735360003842048', '313736399880871936', '313736647499980800', '313738569946648576', '313743560530857

<h2> Examining the ID's with low similarity</h2>

In [44]:
for index, value in enumerate(ids_with_low_similarity): 
    print(index, value) 

0 ['248857355880652800', '@ murekar http://www.trump.com/Hotel_Collection/Trump_Washington_DC/Trump_Washington_DC.asp … http://www.essential-architecture.com/A-AMERICA-N/USA/USA-Washington/DC-032.htm …', '@murekar http://www.trump.com/Hotel_Collection/Trump_Washington_DC/Trump_Washington_DC.asp http://www.essential-architecture.com/A-AMERICA-N/USA/USA-Washington/DC-032.htm', 0.8888888888888888]
1 ['344275492280606720', '"@joooooojaah: I am team @ realDonaldTrump <3 <3 <3" Thanks Georgia!', '"@joooooojaah: I am team @realDonaldTrump &lt;3 &lt;3 &lt;3" Thanks Georgia!', 0.88]
2 ['568521488103403520', 'Via http://examiner.com http://www.examiner.com/article/fox-news-host-urges-donald-trump-to-run-for-president-2016 …', 'Via http://examiner.com http://www.examiner.com/article/fox-news-host-urges-donald-trump-to-run-for-president-2016', 0.8571428571428571]
3 ['1161755131324579840', 'Sad!https://twitter.com/DonaldJTrumpJr/status/1161680797209939968 …', 'Sad!', 0.8888888888888888]
4 ['1174025

<h2> Finding duplicate Rows </h2>

In [58]:
duplicates = mergedData[mergedData.duplicated(['id'])]

print("Duplicate tweets:")
print(duplicates)

Duplicate tweets:
Empty DataFrame
Columns: [id, content, date, metions, hashtags, in_reply_to_user_id_str, is_retweet, retweets, favorites]
Index: []


<p> As we can see here the dataframe of duplicates is all empty. This means we have no duplicates in our data set. </p>

<h2>Testbench for the strip functionality</h2>

Here I examine the strip functionality and the ids with low similiarity

In [46]:
index = 1
s = ids_with_low_similarity[index][1]
s2 = ids_with_low_similarity[index][2]
print(s,"\n", s2)
print(mystrip(s))
print(mystrip(s2))

SequenceMatcher(None, mystrip(s), mystrip(s2)).ratio()

"@joooooojaah: I am team @ realDonaldTrump <3 <3 <3" Thanks Georgia! 
 "@joooooojaah: I am team @realDonaldTrump &lt;3 &lt;3 &lt;3" Thanks Georgia!
"@joooooojaah:iamteam@realdonaldtrump<3<3<3"thanksgeorgia!
"@joooooojaah:iamteam@realdonaldtrump&lt;3&lt;3&lt;3"thanksgeorgia!


0.88